In [1]:
import config
import requests
import datetime as dt
import json
from datetime import datetime as dt, date
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import os
import sys

symbol_list = ['AAPL','MSFT','GOOG','AMZN','NVDA','BRK-B','TSLA','META','JNJ','V']

In [7]:
df_daily = {}
for symbol in symbol_list:
    df_daily[symbol] = pd.read_csv("./stock_price/daily/" + symbol + ".csv")

for symbol in symbol_list:
    df_daily[symbol]['Date'] = pd.to_datetime(df_daily[symbol]['Date'], format='%Y-%m-%d')
    df_daily[symbol]['diff'] = df_daily[symbol]['5. adjusted close'] - df_daily[symbol]['5. adjusted close'].shift(1)
    df_daily[symbol]['direction'] = df_daily[symbol]['diff'].apply(lambda x: 1 if x < 0 else 0)
    df_daily[symbol] = df_daily[symbol][['Date','5. adjusted close','6. volume', 'direction']]
    df_daily[symbol]['Symbol'] = symbol

union_daily = pd.concat([df_daily[symbol] for symbol in symbol_list])
union_daily = union_daily.sort_values(by=['Symbol','Date'])

In [8]:
df_monthly = {}
for symbol in symbol_list:
    df_monthly[symbol] = pd.read_csv("./stock_price/monthly/" + symbol + ".csv")

for symbol in symbol_list:
    df_monthly[symbol]['Date'] = pd.to_datetime(df_monthly[symbol]['Date'], format='%Y-%m-%d')
    df_monthly[symbol]['diff'] = df_monthly[symbol]['5. adjusted close'] - df_monthly[symbol]['5. adjusted close'].shift(1)
    df_monthly[symbol]['direction'] = df_monthly[symbol]['diff'].apply(lambda x: 1 if x < 0 else 0)
    df_monthly[symbol] = df_monthly[symbol][['Date','5. adjusted close','6. volume', 'direction']]
    df_monthly[symbol]['Symbol'] = symbol

union_monthly = pd.concat([df_monthly[symbol] for symbol in symbol_list])
union_monthly = union_monthly.sort_values(by=['Symbol','Date'])

In [9]:
df_weekly = {}
for symbol in symbol_list:
    df_weekly[symbol] = pd.read_csv("./stock_price/monthly/" + symbol + ".csv")

for symbol in symbol_list:
    df_weekly[symbol]['Date'] = pd.to_datetime(df_weekly[symbol]['Date'], format='%Y-%m-%d')
    df_weekly[symbol]['diff'] = df_weekly[symbol]['5. adjusted close'] - df_weekly[symbol]['5. adjusted close'].shift(1)
    df_weekly[symbol]['direction'] = df_weekly[symbol]['diff'].apply(lambda x: 1 if x < 0 else 0)
    df_weekly[symbol] = df_weekly[symbol][['Date','5. adjusted close','6. volume', 'direction']]
    df_weekly[symbol]['Symbol'] = symbol

union_monthly = pd.concat([df_weekly[symbol] for symbol in symbol_list])
union_monthly = union_monthly.sort_values(by=['Symbol','Date'])

In [10]:
if not os.path.isdir("Daily"):
    os.mkdir("Daily")
if not os.path.isdir("Weekly"):
    os.mkdir("Weekly")
if not os.path.isdir("Monthly"):
    os.mkdir("Monthly")

for symbol in symbol_list:
    stock_daily = df_daily[symbol]
    stock_daily.to_csv("Daily/"+symbol+'.csv',index=0)
    stock_weekly = df_weekly[symbol]
    stock_weekly.to_csv("Weekly/"+symbol+'.csv',index=0)
    stock_monthly = df_monthly[symbol]
    stock_monthly.to_csv("Monthly/"+symbol+'.csv',index=0)